In [460]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import  OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression

# Quick Glance

In [461]:
# necessary because 'iteritems' is necessary for plotly and is deprecated for panda > 0
pd.DataFrame.iteritems = pd.DataFrame.items

In [462]:
dataset = pd.read_csv("/Users/antoinebertin/Documents/jedha/full_stack/projects_full_stack/wallmart/Walmart_Store_sales.csv")

In [463]:
# Display the first few rows of the dataframe
dataset.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,18-02-2011,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,25-03-2011,1807545.43,0.0,42.38,3.435,128.616064,7.470
2,17.0,27-07-2012,NaN,0.0,NaN,NaN,130.719581,5.936
3,11.0,NaN,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,28-05-2010,1644470.66,0.0,78.89,2.759,212.412888,7.092


In [464]:
# Get a concise summary of the dataframe
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         150 non-null    float64
 1   Date          132 non-null    object 
 2   Weekly_Sales  136 non-null    float64
 3   Holiday_Flag  138 non-null    float64
 4   Temperature   132 non-null    float64
 5   Fuel_Price    136 non-null    float64
 6   CPI           138 non-null    float64
 7   Unemployment  135 non-null    float64
dtypes: float64(7), object(1)
memory usage: 9.5+ KB


In [465]:
dataset.columns

Index(['Store', 'Date', 'Weekly_Sales', 'Holiday_Flag', 'Temperature',
       'Fuel_Price', 'CPI', 'Unemployment'],
      dtype='object')

# applying some clening

### removing empty cells in 'Weekly_Sales'  - We never use imputation techniques on the target 

In [466]:
dataset['Weekly_Sales'].isna().sum()

14

In [467]:
df_cleaned = dataset.dropna(subset=['Weekly_Sales'])

In [468]:
df_cleaned['Weekly_Sales'].isna().sum()

0

### Converting dates to proper format

In [469]:
df_cleaned['Date'] # its an object

0      18-02-2011
1      25-03-2011
3             NaN
4      28-05-2010
5      28-05-2010
          ...    
145    18-06-2010
146           NaN
147    11-06-2010
148    12-08-2011
149    20-04-2012
Name: Date, Length: 136, dtype: object

In [470]:
# Convert the Date column into datetime format and create new columns
df_cleaned['Date'] = pd.to_datetime(df_cleaned['Date'], errors='coerce')  # Convert Date to datetime, coerce errors to NaT

/var/folders/lt/pb3vc41d15l30zltrtdn76b00000gn/T/ipykernel_7942/2142477784.py:2: UserWarning:

Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.



In [471]:
df_cleaned['Date']

0     2011-02-18
1     2011-03-25
3            NaT
4     2010-05-28
5     2010-05-28
         ...    
145   2010-06-18
146          NaT
147   2010-06-11
148   2011-08-12
149   2012-04-20
Name: Date, Length: 136, dtype: datetime64[ns]

In [472]:
df_cleaned['Year'] = df_cleaned['Date'].dt.year
df_cleaned['Month'] = df_cleaned['Date'].dt.month
df_cleaned['Day'] = df_cleaned['Date'].dt.day
df_cleaned['DayOfWeek'] = df_cleaned['Date'].dt.dayofweek

In [473]:
df_cleaned = df_cleaned.drop(columns=['Date'])
df_cleaned.head()

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Day,DayOfWeek
0,6.0,1572117.54,NaN,59.61,3.045,214.777523,6.858,2011.0,2.0,18.0,4.0
1,13.0,1807545.43,0.0,42.38,3.435,128.616064,7.470,2011.0,3.0,25.0,4.0
3,11.0,1244390.03,0.0,84.57,NaN,214.556497,7.346,NaN,NaN,NaN,NaN
4,6.0,1644470.66,0.0,78.89,2.759,212.412888,7.092,2010.0,5.0,28.0,4.0
5,4.0,1857533.70,0.0,NaN,2.756,126.160226,7.896,2010.0,5.0,28.0,4.0


In [474]:
df_cleaned = df_cleaned.dropna(subset=['Year', 'Month', 'Day', 'DayOfWeek'])

In [475]:
# Drop rows with missing Year, Month, Day, DayOfWeek due to Date conversion errors
df_cleaned.isna().sum()

Store            0
Weekly_Sales     0
Holiday_Flag     9
Temperature     11
Fuel_Price      11
CPI              9
Unemployment    11
Year             0
Month            0
Day              0
DayOfWeek        0
dtype: int64

### Remove Outliers -> +/- 3 std

In [476]:
df_cleaned.count()

Store           118
Weekly_Sales    118
Holiday_Flag    109
Temperature     107
Fuel_Price      107
CPI             109
Unemployment    107
Year            118
Month           118
Day             118
DayOfWeek       118
dtype: int64

In [477]:
higher = df_cleaned['Unemployment'].mean() + (3*df_cleaned['Unemployment'].std())
lower = df_cleaned['Unemployment'].mean() - (3*df_cleaned['Unemployment'].std())
higher, lower

(12.715763077165814, 2.646835053675309)

In [478]:
df_cleaned[dataset['Unemployment']>higher]
#df_cleaned[dataset['Unemployment']<lower] # None

/var/folders/lt/pb3vc41d15l30zltrtdn76b00000gn/T/ipykernel_7942/3691024032.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Day,DayOfWeek
30,12.0,1021154.48,0.0,68.40,4.193,129.044433,13.736,2011.0,5.0,6.0,4.0
62,12.0,1295605.35,0.0,52.77,3.236,126.879484,14.313,2010.0,12.0,17.0,4.0
68,12.0,955913.68,0.0,91.04,3.701,129.201581,13.503,2011.0,8.0,12.0,4.0
93,12.0,964332.51,0.0,NaN,4.087,129.062355,13.736,2011.0,5.0,27.0,4.0
135,12.0,903119.03,1.0,83.63,3.044,126.114581,14.180,2010.0,9.0,10.0,4.0


In [479]:
# Calculate mean and standard deviation for each column to identify outliers
outlier_columns = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment']
for column in outlier_columns:
    mean = df_cleaned[column].mean()
    std = df_cleaned[column].std()
    lower_bound = mean - 3*std
    upper_bound = mean + 3*std
    
    # Drop rows that are considered outliers based on the calculated bounds
    df_cleaned = df_cleaned[(df_cleaned[column] >= lower_bound) & (df_cleaned[column] <= upper_bound)]

In [480]:
df_cleaned.count()

Store           80
Weekly_Sales    80
Holiday_Flag    71
Temperature     80
Fuel_Price      80
CPI             80
Unemployment    80
Year            80
Month           80
Day             80
DayOfWeek       80
dtype: int64

### Impute missing value for holiday_flag with mode

In [481]:
df_cleaned.isna().sum() # lets work in imputing

Store           0
Weekly_Sales    0
Holiday_Flag    9
Temperature     0
Fuel_Price      0
CPI             0
Unemployment    0
Year            0
Month           0
Day             0
DayOfWeek       0
dtype: int64

In [482]:
df_cleaned['Holiday_Flag'].value_counts()

Holiday_Flag
0.0    65
1.0     6
Name: count, dtype: int64

In [483]:
holiday_flag_mode = df_cleaned['Holiday_Flag'].mode()[0] # its a binary so I ll chose the mode to impute with the most frequent value used
holiday_flag_mode

0.0

In [484]:
df_cleaned['Holiday_Flag'].fillna(holiday_flag_mode, inplace=True)

In [485]:
df_cleaned['Holiday_Flag'].isna().sum()

0

# Split and transform

In [486]:
df_cleaned.head()

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Day,DayOfWeek
0,6.0,1572117.54,0.0,59.61,3.045,214.777523,6.858,2011.0,2.0,18.0,4.0
1,13.0,1807545.43,0.0,42.38,3.435,128.616064,7.470,2011.0,3.0,25.0,4.0
4,6.0,1644470.66,0.0,78.89,2.759,212.412888,7.092,2010.0,5.0,28.0,4.0
6,15.0,695396.19,0.0,69.80,4.069,134.855161,7.658,2011.0,6.0,3.0,4.0
7,20.0,2203523.20,0.0,39.93,3.617,213.023622,6.961,2012.0,2.0,3.0,4.0


In [487]:
X = df_cleaned.drop('Weekly_Sales', axis=1)
y = df_cleaned[['Weekly_Sales']]

In [488]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [489]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((64, 10), (16, 10), (64, 1), (16, 1))

In [490]:
# Create pipeline for numeric features
numeric_features = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Year', 'Month', 'Day', 'DayOfWeek']
numeric_transformer = Pipeline(
    steps=[
    ('scaler', StandardScaler())
])

# Create pipeline for categorical features
categorical_features= ['Store', 'Holiday_Flag']
categorical_transformer = Pipeline(
    steps=[
    ('encoder', OneHotEncoder(drop='first')) # first column will be dropped to avoid creating correlations between features
    ])

In [491]:
# Use ColumnTransformer to make a preprocessor object that describes all the treatments to be done
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [492]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

# Train model and R2

In [493]:
# Train model
model = LinearRegression()

model.fit(X_train, y_train)  # Training is always done on train set !!

# Predictions on training set
Y_train_pred = model.predict(X_train)
# Predictions on test set
Y_test_pred = model.predict(X_test)

In [494]:
# Print R^2 scores
print("R2 score on training set : ", model.score(X_train, y_train))
print("R2 score on test set : ", model.score(X_test, y_test))

R2 score on training set :  0.9820943694655776
R2 score on test set :  0.9637322357608794


In [495]:
len(preprocessor.get_feature_names_out()) # I used 27 columns, lets see if I can do the same after reg with less columns

27

# feature_importance_df of the base model

In [496]:
feature_names_transformed = preprocessor.get_feature_names_out() #because of the hot-encode I need to do it that way

In [497]:
# Create a DataFrame with feature names and their corresponding coefficients
feature_importance_df = pd.DataFrame({
    'Feature': feature_names_transformed,
    'Coefficient': model.coef_.flatten()  # Ensure the coefficients are a flat array
})

# Calculate the absolute value of coefficients to understand their impact
feature_importance_df['Absolute_Coefficient'] = feature_importance_df['Coefficient'].abs()

# Sort the features by the absolute value of their coefficient in descending order
feature_importance_df = feature_importance_df.sort_values(by='Absolute_Coefficient', ascending=False)

# Reset index for better readability
feature_importance_df.reset_index(drop=True, inplace=True)

In [498]:
feature_importance_df

,Feature,Coefficient,Absolute_Coefficient
0,cat__Store_4.0,1.767632e+06,1.767632e+06
1,cat__Store_10.0,1.691010e+06,1.691010e+06
2,cat__Store_13.0,1.602475e+06,1.602475e+06
3,cat__Store_5.0,-1.395599e+06,1.395599e+06
4,cat__Store_3.0,-1.291081e+06,1.291081e+06
5,cat__Store_9.0,-1.244564e+06,1.244564e+06
6,cat__Store_19.0,9.988863e+05,9.988863e+05
7,cat__Store_14.0,9.646134e+05,9.646134e+05
8,cat__Store_8.0,-8.200410e+05,8.200410e+05
9,cat__Store_16.0,-8.009231e+05,8.009231e+05


# regularized the base model with Lasso

In [499]:
from sklearn.linear_model import LassoCV
import numpy as np

In [511]:
#alphas = np.logspace(-3, 3, 5)
alphas = np.logspace(-6, 6, 13)

In [512]:
lasso_cv = LassoCV(alphas=alphas, cv=5, random_state=42, max_iter=50000)

In [513]:
# Fit LassoCV model
lasso_cv.fit(X_train, y_train.values.ravel())

LassoCV(alphas=array([1.e-06, 1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01,
       1.e+02, 1.e+03, 1.e+04, 1.e+05, 1.e+06]),
        cv=5, max_iter=50000, random_state=42)

In [514]:
# Print R^2 scores
print("R2 score on training set : ", lasso_cv.score(X_train, y_train))
print("R2 score on test set : ", lasso_cv.score(X_test, y_test))

R2 score on training set :  0.9820649871300755
R2 score on test set :  0.9621162542005425


In [529]:
# Count of non-zero (used) features
np.sum(lasso_cv.coef_ == 0) # so I have removed 2 columns

2

In [518]:
lasso_cv.alpha_ # best alpha from CV search

10.0

In [535]:
feature_names_lasso = preprocessor.get_feature_names_out() #because of the hot-encode I need to do it that way

In [536]:
# Create a DataFrame with feature names and their corresponding coefficients
feature_importance_lasso_df = pd.DataFrame({
    'Feature': feature_names_lasso,
    'Coefficient': lasso_cv.coef_.flatten()  # Ensure the coefficients are a flat array
})

# Calculate the absolute value of coefficients to understand their impact
feature_importance_lasso_df['Absolute_Coefficient'] = feature_importance_lasso_df['Coefficient'].abs()

# Sort the features by the absolute value of their coefficient in descending order
feature_importance_lasso_df = feature_importance_lasso_df.sort_values(by='Absolute_Coefficient', ascending=True)

# Reset index for better readability
feature_importance_lasso_df.reset_index(drop=True, inplace=True)

In [540]:
feature_importance_lasso_df.head() #I can see that I can remove the first two

,Feature,Coefficient,Absolute_Coefficient
0,cat__Store_15.0,-0.000000,0.000000
1,num__DayOfWeek,0.000000,0.000000
2,cat__Store_17.0,-23359.919272,23359.919272
3,num__Day,-27044.792783,27044.792783
4,num__Year,27481.703523,27481.703523


# Run Lasso with its best alpha and with less column than the base model

In [560]:
X_train_reduced, X_test_reduced, y_train_reduced, y_test_reduced = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [561]:
X_train_reduced = preprocessor.fit_transform(X_train_reduced)
X_test_reduced = preprocessor.transform(X_test_reduced)

In [562]:
X_train_transformed_df = pd.DataFrame(X_train_reduced, columns=preprocessor.get_feature_names_out())
X_test_transformed_df = pd.DataFrame(X_test_reduced, columns=preprocessor.get_feature_names_out())

In [564]:
columns_to_keep = [col for col in preprocessor.get_feature_names_out() if col not in ['cat__Store_15.0', 'num__DayOfWeek']]
len(columns_to_keep)

25

In [565]:
X_train_reduced = X_train_transformed_df[columns_to_keep]
X_test_reduced = X_test_transformed_df[columns_to_keep]

In [ ]:
# X_train_reduced_np = X_train_reduced.to_numpy()
# X_test_reduced_np = X_test_reduced.to_numpy()

In [566]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Lasso

In [572]:
# Initialize Lasso model with the best alpha from LassoCV
lasso_best = Lasso(alpha=10, max_iter=50000, random_state=42)
# Perform cross-validation to get R^2 scores
r2_scores_train = cross_val_score(lasso_best, X_train_reduced, y_train_reduced.values.ravel(), cv=5, scoring='r2')
r2_scores_test = cross_val_score(lasso_best, X_test_reduced, y_test_reduced.values.ravel(), cv=5, scoring='r2')

In [573]:
r2_scores_train.mean(), r2_scores_train.std()

(0.8771219886485653, 0.11573223091433858)

In [574]:
r2_scores_test.mean(), r2_scores_test.std()

(0.08255277718692897, 0.38944025114429426)